In [500]:
import pandas as pd
import os
import sklearn
import numpy as np
import pandas as pd
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
from sklearn import tree, metrics
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


cleaning of data

In [565]:
fifa_21=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/ai/players_21.csv")

In [566]:
missing_values = fifa_21.isna().sum() / fifa_21.shape[0] * 100
columns_to_drop = missing_values[missing_values > 30].index.tolist()
fifa_21 = fifa_21.drop(columns_to_drop, axis=1)
fifa_21.to_csv('dataset_with_missing_values_removed.csv', index=False)
fifa_21





,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,...,lb,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_flag_url
0,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,"RW, ST, CF",93,93,103500000.0,560000.0,33,...,62+3,52+3,52+3,52+3,62+3,19+3,https://cdn.sofifa.net/players/158/023/21_120.png,https://cdn.sofifa.net/teams/241/60.png,https://cdn.sofifa.net/flags/es.png,https://cdn.sofifa.net/flags/ar.png
1,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"ST, LW",92,92,63000000.0,220000.0,35,...,61+3,54+3,54+3,54+3,61+3,20+3,https://cdn.sofifa.net/players/020/801/21_120.png,https://cdn.sofifa.net/teams/45/60.png,https://cdn.sofifa.net/flags/it.png,https://cdn.sofifa.net/flags/pt.png
2,188545,https://sofifa.com/player/188545/robert-lewand...,R. Lewandowski,Robert Lewandowski,ST,91,91,111000000.0,240000.0,31,...,61+3,60+3,60+3,60+3,61+3,19+3,https://cdn.sofifa.net/players/188/545/21_120.png,https://cdn.sofifa.net/teams/21/60.png,https://cdn.sofifa.net/flags/de.png,https://cdn.sofifa.net/flags/pl.png
3,190871,https://sofifa.com/player/190871/neymar-da-sil...,Neymar Jr,Neymar da Silva Santos Júnior,"LW, CAM",91,91,132000000.0,270000.0,28,...,62+3,49+3,49+3,49+3,62+3,20+3,https://cdn.sofifa.net/players/190/871/21_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,https://cdn.sofifa.net/flags/br.png
4,192985,https://sofifa.com/player/192985/kevin-de-bruy...,K. De Bruyne,Kevin De Bruyne,"CAM, CM",91,91,129000000.0,370000.0,29,...,75+3,69+3,69+3,69+3,75+3,21+3,https://cdn.sofifa.net/players/192/985/21_120.png,https://cdn.sofifa.net/teams/10/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/flags/be.png
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18939,257710,https://sofifa.com/player/257710/mengxuan-zhan...,Zhang Mengxuan,张梦炫,CB,47,52,70000.0,1000.0,21,...,45+2,47+2,47+2,47+2,45+2,15+2,https://cdn.sofifa.net/players/257/710/21_120.png,https://cdn.sofifa.net/teams/112165/60.png,https://cdn.sofifa.net/flags/cn.png,https://cdn.sofifa.net/flags/cn.png
18940,257933,https://sofifa.com/player/257933/wenzhuo-huang...,Huang Wenzhou,黄文卓,CM,47,53,70000.0,1000.0,21,...,47+2,46+2,46+2,46+2,47+2,15+2,https://cdn.sofifa.net/players/257/933/21_120.png,https://cdn.sofifa.net/teams/112540/60.png,https://cdn.sofifa.net/flags/cn.png,https://cdn.sofifa.net/flags/cn.png
18941,257936,https://sofifa.com/player/257936/yue-song/210002,Song Yue,宋岳,CM,47,47,45000.0,2000.0,28,...,47,46+1,46+1,46+1,47,11+2,https://cdn.sofifa.net/players/257/936/21_120.png,https://cdn.sofifa.net/teams/111774/60.png,https://cdn.sofifa.net/flags/cn.png,https://cdn.sofifa.net/flags/cn.png
18942,258736,https://sofifa.com/player/258736/vani-da-silva...,V. Da Silva,Ivanilson Loforte Tique Da Silva,ST,47,67,130000.0,500.0,17,...,33+2,26+2,26+2,26+2,33+2,14+2,https://cdn.sofifa.net/players/258/736/21_120.png,https://cdn.sofifa.net/teams/1920/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/flags/gb-eng.png


In [567]:
correlation_matrix = fifa_21.corr()

<ipython-input-567-4f3a38e95ff5>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  correlation_matrix = fifa_21.corr()


feature selection

In [615]:
selected_features= correlation_matrix['overall'].sort_values(ascending=False).index
selected_features[:10]


Index(['overall', 'movement_reactions', 'passing', 'mentality_composure',
       'dribbling', 'potential', 'release_clause_eur', 'wage_eur', 'value_eur',
       'power_shot_power'],
      dtype='object')

In [616]:
selected_features=[ 'overall', 'movement_reactions', 'passing', 'mentality_composure',
       'dribbling', 'potential', 'release_clause_eur', 'wage_eur', 'value_eur',
       'power_shot_power']

In [618]:
new_df = selected_features.copy()
new_fifa=pd.DataFrame(fifa_21[new_df])
new_fifa

,overall,movement_reactions,passing,mentality_composure,dribbling,potential,release_clause_eur,wage_eur,value_eur,power_shot_power
0,93,94,91.0,96,95.0,93,138400000.0,560000.0,103500000.0,86
1,92,95,81.0,95,89.0,92,75900000.0,220000.0,63000000.0,94
2,91,93,78.0,88,85.0,91,132000000.0,240000.0,111000000.0,89
3,91,91,86.0,93,94.0,91,166500000.0,270000.0,132000000.0,80
4,91,91,93.0,91,88.0,91,161000000.0,370000.0,129000000.0,91
...,...,...,...,...,...,...,...,...,...,...
18939,47,48,26.0,40,27.0,52,57000.0,1000.0,70000.0,35
18940,47,50,49.0,35,47.0,53,72000.0,1000.0,70000.0,38
18941,47,44,49.0,35,46.0,47,47000.0,2000.0,45000.0,49
18942,47,53,40.0,45,53.0,67,165000.0,500.0,130000.0,50


In [573]:


new_fifa = new_fifa.fillna(new_fifa.mean())
new_fifa




,overall,movement_reactions,passing,mentality_composure,dribbling,potential,release_clause_eur,wage_eur,value_eur,power_shot_power,physic,mentality_vision,attacking_short_passing,shooting,skill_long_passing
0,93,94,91.0,96,95.0,93,138400000.0,560000.0,103500000.0,86,65.0,95,91,92.0,91
1,92,95,81.0,95,89.0,92,75900000.0,220000.0,63000000.0,94,77.0,82,82,93.0,77
2,91,93,78.0,88,85.0,91,132000000.0,240000.0,111000000.0,89,82.0,79,84,91.0,70
3,91,91,86.0,93,94.0,91,166500000.0,270000.0,132000000.0,80,59.0,90,87,85.0,81
4,91,91,93.0,91,88.0,91,161000000.0,370000.0,129000000.0,91,78.0,94,94,86.0,93
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18939,47,48,26.0,40,27.0,52,57000.0,1000.0,70000.0,35,48.0,25,31,23.0,25
18940,47,50,49.0,35,47.0,53,72000.0,1000.0,70000.0,38,48.0,53,55,32.0,50
18941,47,44,49.0,35,46.0,47,47000.0,2000.0,45000.0,49,59.0,44,56,37.0,56
18942,47,53,40.0,45,53.0,67,165000.0,500.0,130000.0,50,40.0,49,45,46.0,37


In [574]:
new_fifa

,overall,movement_reactions,passing,mentality_composure,dribbling,potential,release_clause_eur,wage_eur,value_eur,power_shot_power,physic,mentality_vision,attacking_short_passing,shooting,skill_long_passing
0,93,94,91.0,96,95.0,93,138400000.0,560000.0,103500000.0,86,65.0,95,91,92.0,91
1,92,95,81.0,95,89.0,92,75900000.0,220000.0,63000000.0,94,77.0,82,82,93.0,77
2,91,93,78.0,88,85.0,91,132000000.0,240000.0,111000000.0,89,82.0,79,84,91.0,70
3,91,91,86.0,93,94.0,91,166500000.0,270000.0,132000000.0,80,59.0,90,87,85.0,81
4,91,91,93.0,91,88.0,91,161000000.0,370000.0,129000000.0,91,78.0,94,94,86.0,93
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18939,47,48,26.0,40,27.0,52,57000.0,1000.0,70000.0,35,48.0,25,31,23.0,25
18940,47,50,49.0,35,47.0,53,72000.0,1000.0,70000.0,38,48.0,53,55,32.0,50
18941,47,44,49.0,35,46.0,47,47000.0,2000.0,45000.0,49,59.0,44,56,37.0,56
18942,47,53,40.0,45,53.0,67,165000.0,500.0,130000.0,50,40.0,49,45,46.0,37


In [575]:
row_index = 0
new_fifa.loc[row_index, 'overall'] = 92
new_fifa.to_csv('updated_dataset.csv', index=False)
new_fifa


,overall,movement_reactions,passing,mentality_composure,dribbling,potential,release_clause_eur,wage_eur,value_eur,power_shot_power,physic,mentality_vision,attacking_short_passing,shooting,skill_long_passing
0,92,94,91.0,96,95.0,93,138400000.0,560000.0,103500000.0,86,65.0,95,91,92.0,91
1,92,95,81.0,95,89.0,92,75900000.0,220000.0,63000000.0,94,77.0,82,82,93.0,77
2,91,93,78.0,88,85.0,91,132000000.0,240000.0,111000000.0,89,82.0,79,84,91.0,70
3,91,91,86.0,93,94.0,91,166500000.0,270000.0,132000000.0,80,59.0,90,87,85.0,81
4,91,91,93.0,91,88.0,91,161000000.0,370000.0,129000000.0,91,78.0,94,94,86.0,93
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18939,47,48,26.0,40,27.0,52,57000.0,1000.0,70000.0,35,48.0,25,31,23.0,25
18940,47,50,49.0,35,47.0,53,72000.0,1000.0,70000.0,38,48.0,53,55,32.0,50
18941,47,44,49.0,35,46.0,47,47000.0,2000.0,45000.0,49,59.0,44,56,37.0,56
18942,47,53,40.0,45,53.0,67,165000.0,500.0,130000.0,50,40.0,49,45,46.0,37


In [576]:
data=new_fifa.copy()

In [577]:
data

,overall,movement_reactions,passing,mentality_composure,dribbling,potential,release_clause_eur,wage_eur,value_eur,power_shot_power,physic,mentality_vision,attacking_short_passing,shooting,skill_long_passing
0,92,94,91.0,96,95.0,93,138400000.0,560000.0,103500000.0,86,65.0,95,91,92.0,91
1,92,95,81.0,95,89.0,92,75900000.0,220000.0,63000000.0,94,77.0,82,82,93.0,77
2,91,93,78.0,88,85.0,91,132000000.0,240000.0,111000000.0,89,82.0,79,84,91.0,70
3,91,91,86.0,93,94.0,91,166500000.0,270000.0,132000000.0,80,59.0,90,87,85.0,81
4,91,91,93.0,91,88.0,91,161000000.0,370000.0,129000000.0,91,78.0,94,94,86.0,93
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18939,47,48,26.0,40,27.0,52,57000.0,1000.0,70000.0,35,48.0,25,31,23.0,25
18940,47,50,49.0,35,47.0,53,72000.0,1000.0,70000.0,38,48.0,53,55,32.0,50
18941,47,44,49.0,35,46.0,47,47000.0,2000.0,45000.0,49,59.0,44,56,37.0,56
18942,47,53,40.0,45,53.0,67,165000.0,500.0,130000.0,50,40.0,49,45,46.0,37


In [608]:
from sklearn.preprocessing import StandardScaler as sc

scaler = StandardScaler()

training and testing

In [609]:
y=new_fifa['overall']


In [614]:
#X=new_fifa[['overall','potential']]
X=sc.fit_transform(new_fifa)
X=pd.DataFrame(X,colums=new_fifa.colums)



TypeError: ignored

In [580]:
Xtrain,Xtest,Ytrain,Ytest=train_test_split(X,y,test_size=0.2,random_state=42,stratify=y)

In [581]:
from sklearn.ensemble import RandomForestRegressor


In [582]:
rf = RandomForestRegressor()
rf.fit(Xtrain, Ytrain)



RandomForestRegressor()

In [583]:
from sklearn.metrics import accuracy_score

In [584]:
y_pred=rf.predict(Xtest)
y_pred

array([67., 65., 77., ..., 74., 60., 62.])

In [586]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [587]:
mean_absolute_error(y_pred,Ytest)

0.00019530219055159537

In [588]:
from sklearn.tree import DecisionTreeRegressor


In [589]:
dt = DecisionTreeRegressor()
dt.fit(Xtrain, Ytrain)

DecisionTreeRegressor()

In [590]:

y_pred = dt.predict(Xtest)
y_pred



array([67., 65., 77., ..., 74., 60., 62.])

In [591]:
mae=mean_squared_error(y_pred,Ytest)
mae

0.0002639218791237794

In [592]:
from xgboost import XGBRegressor


In [600]:
xgb = XGBRegressor()


In [601]:
xgb.fit(Xtrain, Ytrain)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [602]:
y_pred = xgb.predict(Xtest)


In [603]:
y_pred

array([66.99999 , 65.      , 76.99999 , ..., 74.00001 , 60.000004,
       62.000004], dtype=float32)

In [604]:
mse = mean_absolute_error(Ytest, y_pred)
mse

1.2792172989451372e-05

optimization and fine tuning

In [605]:
from sklearn.model_selection import RandomizedSearchCV
param_distributions = {
    'n_estimators': np.arange(10, 100, step=10),
    'max_depth': np.arange(2, 10, step=1)
}
n_iter = 100
random_search = RandomizedSearchCV(
    estimator=RandomForestRegressor(),
    param_distributions=param_distributions,
    n_iter=n_iter,
    cv=5
)
random_search.fit(Xtrain, Ytrain)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:305: UserWarning: The total space of parameters 72 is smaller than n_iter=100. Running 72 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(), n_iter=100,
                   param_distributions={'max_depth': array([2, 3, 4, 5, 6, 7, 8, 9]),
                                        'n_estimators': array([10, 20, 30, 40, 50, 60, 70, 80, 90])})

In [454]:
best_params = random_search.best_params_
best_rf = RandomForestRegressor(**best_params)
best_rf.fit(Xtrain, Ytrain)

RandomForestRegressor(max_depth=9, n_estimators=50)

In [455]:
y_pred = best_rf.predict(Xtest)
mse = mean_squared_error(Ytest, y_pred)
mse

3.4415413037741686e-05

In [456]:
param_distributions = {
    'max_depth': np.arange(2, 10, step=1),
    'min_samples_split': np.arange(2, 100, step=10),
    'max_features': np.arange(1, 10, step=1)
}
n_iter = 100
random_search = RandomizedSearchCV(
    estimator=DecisionTreeRegressor(),
    param_distributions=param_distributions,
    n_iter=n_iter,
    cv=5
)
random_search.fit(Xtrain, Ytrain)


RandomizedSearchCV(cv=5, estimator=DecisionTreeRegressor(), n_iter=100,
                   param_distributions={'max_depth': array([2, 3, 4, 5, 6, 7, 8, 9]),
                                        'max_features': array([1, 2, 3, 4, 5, 6, 7, 8, 9]),
                                        'min_samples_split': array([ 2, 12, 22, 32, 42, 52, 62, 72, 82, 92])})

In [457]:
best_params = random_search.best_params_
best_dt = DecisionTreeRegressor(**best_params)
best_dt.fit(Xtrain, Ytrain)

DecisionTreeRegressor(max_depth=8, max_features=8)

In [458]:
y_pred = best_dt.predict(Xtest)
mean_squared_error(Ytest, y_pred)

0.0002639218791237794

In [459]:
param_distributions = {
    'n_estimators': np.arange(100, 500, step=100),
    'max_depth': np.arange(2, 10, step=1),
    'learning_rate': np.arange(0.01, 0.1, step=0.01),
    'gamma': np.arange(0, 1, step=0.1),
    'subsample': np.arange(0.5, 1, step=0.1),
    'colsample_bytree': np.arange(0.5, 1, step=0.1)
}
n_iter = 100
random_search = RandomizedSearchCV(
    estimator=XGBRegressor(),
    param_distributions=param_distributions,
    n_iter=n_iter,
    cv=5
)
random_search.fit(Xtrain, Ytrain)





RandomizedSearchCV(cv=5,
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None, device=None,
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric=None, feature_types=None,
                                          gamma=None, grow_policy=None,
                                          importance_type=None,
                                          interaction_constraints=None,
                                          learning_rate=...
                                          num_parallel_tree=None,
                                          random_state=None, ...),
                   n_iter=100,
                   param_distributions={'colsample_bytree': array([0.5, 0.6, 0.7, 0.8, 0.9]),
                                        'gamma': array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]),
                                        'learning_rate': array([0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09]),
                                        'max_depth': array([2, 3, 4, 5, 6, 7, 8, 9]),
                                        'n_estimators': array([100, 200, 300, 400]),
                                        'subsample': array([0.5, 0.6, 0.7, 0.8, 0.9])})

In [390]:
best_params = random_search.best_params_
best_xgb = XGBRegressor(**best_params)
best_xgb.fit(Xtrain, Ytrain)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8999999999999999, device=None,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=0.2, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.09, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=5,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=400,
             n_jobs=None, num_parallel_tree=None, random_state=None, ...)

In [391]:
y_pred = best_xgb.predict(Xtest)
mean_squared_error(Ytest, y_pred)


0.0014304562237447813

In [392]:
#COMPARING WITH FIFA 2022
fifa_22=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/ai/players_22.csv")
fifa_22

<ipython-input-392-ef8ba59be450>:2: DtypeWarning: Columns (25,108) have mixed types. Specify dtype option on import or set low_memory=False.
  fifa_22=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/ai/players_22.csv")


,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,...,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url
0,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,"RW, ST, CF",93,93,78000000.0,320000.0,34,...,50+3,50+3,50+3,61+3,19+3,https://cdn.sofifa.net/players/158/023/22_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,https://cdn.sofifa.net/teams/1369/60.png,https://cdn.sofifa.net/flags/ar.png
1,188545,https://sofifa.com/player/188545/robert-lewand...,R. Lewandowski,Robert Lewandowski,ST,92,92,119500000.0,270000.0,32,...,60+3,60+3,60+3,61+3,19+3,https://cdn.sofifa.net/players/188/545/22_120.png,https://cdn.sofifa.net/teams/21/60.png,https://cdn.sofifa.net/flags/de.png,https://cdn.sofifa.net/teams/1353/60.png,https://cdn.sofifa.net/flags/pl.png
2,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"ST, LW",91,91,45000000.0,270000.0,36,...,53+3,53+3,53+3,60+3,20+3,https://cdn.sofifa.net/players/020/801/22_120.png,https://cdn.sofifa.net/teams/11/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1354/60.png,https://cdn.sofifa.net/flags/pt.png
3,190871,https://sofifa.com/player/190871/neymar-da-sil...,Neymar Jr,Neymar da Silva Santos Júnior,"LW, CAM",91,91,129000000.0,270000.0,29,...,50+3,50+3,50+3,62+3,20+3,https://cdn.sofifa.net/players/190/871/22_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,NaN,https://cdn.sofifa.net/flags/br.png
4,192985,https://sofifa.com/player/192985/kevin-de-bruy...,K. De Bruyne,Kevin De Bruyne,"CM, CAM",91,91,125500000.0,350000.0,30,...,69+3,69+3,69+3,75+3,21+3,https://cdn.sofifa.net/players/192/985/22_120.png,https://cdn.sofifa.net/teams/10/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1325/60.png,https://cdn.sofifa.net/flags/be.png
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19234,261962,https://sofifa.com/player/261962/defu-song/220002,Song Defu,宋德福,CDM,47,52,70000.0,1000.0,22,...,46+2,46+2,46+2,48+2,15+2,https://cdn.sofifa.net/players/261/962/22_120.png,https://cdn.sofifa.net/teams/112541/60.png,https://cdn.sofifa.net/flags/cn.png,NaN,https://cdn.sofifa.net/flags/cn.png
19235,262040,https://sofifa.com/player/262040/caoimhin-port...,C. Porter,Caoimhin Porter,CM,47,59,110000.0,500.0,19,...,44+2,44+2,44+2,48+2,14+2,https://cdn.sofifa.net/players/262/040/22_120.png,https://cdn.sofifa.net/teams/445/60.png,https://cdn.sofifa.net/flags/ie.png,NaN,https://cdn.sofifa.net/flags/ie.png
19236,262760,https://sofifa.com/player/262760/nathan-logue/...,N. Logue,Nathan Logue-Cunningham,CM,47,55,100000.0,500.0,21,...,45+2,45+2,45+2,47+2,12+2,https://cdn.sofifa.net/players/262/760/22_120.png,https://cdn.sofifa.net/teams/111131/60.png,https://cdn.sofifa.net/flags/ie.png,NaN,https://cdn.sofifa.net/flags/ie.png
19237,262820,https://sofifa.com/player/262820/luke-rudden/2...,L. Rudden,Luke Rudden,ST,47,60,110000.0,500.0,19,...,26+2,26+2,26+2,32+2,15+2,https://cdn.sofifa.net/players/262/820/22_120.png,https://cdn.sofifa.net/teams/111131/60.png,https://cdn.sofifa.net/flags/ie.png,NaN,https://cdn.sofifa.net/flags/ie.png


In [393]:
correlation_matrix = fifa_22.corr()

<ipython-input-393-c5df40832efd>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  correlation_matrix = fifa_22.corr()


In [394]:
features_22= correlation_matrix['overall'].sort_values(ascending=False).index
features_22[:40]

Index(['overall', 'movement_reactions', 'passing', 'mentality_composure',
       'dribbling', 'potential', 'wage_eur', 'power_shot_power', 'value_eur',
       'release_clause_eur', 'physic', 'mentality_vision',
       'attacking_short_passing', 'goalkeeping_speed', 'skill_long_passing',
       'shooting', 'international_reputation', 'skill_ball_control', 'age',
       'skill_curve', 'power_long_shots', 'attacking_crossing',
       'mentality_aggression', 'power_stamina', 'skill_dribbling',
       'skill_fk_accuracy', 'attacking_volleys', 'skill_moves',
       'power_strength', 'mentality_positioning', 'defending',
       'attacking_heading_accuracy', 'mentality_penalties',
       'attacking_finishing', 'mentality_interceptions',
       'defending_marking_awareness', 'power_jumping', 'movement_agility',
       'defending_standing_tackle', 'defending_sliding_tackle'],
      dtype='object')

In [395]:
features_22=['overall', 'movement_reactions', 'passing', 'mentality_composure',
       'dribbling', 'potential', 'wage_eur', 'power_shot_power', 'value_eur',
       'release_clause_eur', 'physic', 'mentality_vision',
       'attacking_short_passing', 'goalkeeping_speed', 'skill_long_passing',
       'shooting', 'international_reputation', 'skill_ball_control', 'age',
       'skill_curve', 'power_long_shots', 'attacking_crossing',
       'mentality_aggression', 'power_stamina', 'skill_dribbling',
       'skill_fk_accuracy', 'attacking_volleys', 'skill_moves',
       'power_strength', 'mentality_positioning', 'defending',
       'attacking_heading_accuracy', 'mentality_penalties',
       'attacking_finishing', 'mentality_interceptions',
       'defending_marking_awareness', 'power_jumping', 'movement_agility',
       'defending_standing_tackle', 'defending_sliding_tackle']

In [396]:
df_22 = selected_features.copy()
newff_22=pd.DataFrame(fifa_22[df_22])
newff_22

,overall,movement_reactions,passing,mentality_composure,dribbling,potential,wage_eur,power_shot_power,value_eur,release_clause_eur,...,defending,attacking_heading_accuracy,mentality_penalties,attacking_finishing,mentality_interceptions,defending_marking_awareness,power_jumping,movement_agility,defending_standing_tackle,defending_sliding_tackle
0,93,94,91.0,96,95.0,93,320000.0,86,78000000.0,144300000.0,...,34.0,70,75,95,40,20,68,91,35,24
1,92,93,79.0,88,86.0,92,270000.0,90,119500000.0,197200000.0,...,44.0,90,90,95,49,35,85,77,42,19
2,91,94,80.0,95,88.0,91,270000.0,94,45000000.0,83300000.0,...,34.0,90,88,95,29,24,95,86,32,24
3,91,89,86.0,93,94.0,91,270000.0,80,129000000.0,238700000.0,...,37.0,63,93,83,37,35,64,96,32,29
4,91,91,93.0,89,88.0,91,350000.0,91,125500000.0,232200000.0,...,64.0,55,83,82,66,68,63,79,65,53
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19234,47,53,46.0,37,48.0,52,1000.0,46,70000.0,114000.0,...,42.0,48,42,32,42,38,62,55,43,48
19235,47,49,50.0,47,46.0,59,500.0,49,110000.0,193000.0,...,41.0,46,44,33,39,37,47,64,44,47
19236,47,46,45.0,36,49.0,55,500.0,50,100000.0,175000.0,...,41.0,43,39,32,38,38,57,58,44,48
19237,47,48,36.0,47,48.0,60,500.0,48,110000.0,239000.0,...,15.0,40,49,49,14,10,50,72,14,11


In [397]:
newff_22 = newff_22.fillna(newff_22.mean())
newff_22

,overall,movement_reactions,passing,mentality_composure,dribbling,potential,wage_eur,power_shot_power,value_eur,release_clause_eur,...,defending,attacking_heading_accuracy,mentality_penalties,attacking_finishing,mentality_interceptions,defending_marking_awareness,power_jumping,movement_agility,defending_standing_tackle,defending_sliding_tackle
0,93,94,91.0,96,95.0,93,320000.0,86,78000000.0,144300000.0,...,34.0,70,75,95,40,20,68,91,35,24
1,92,93,79.0,88,86.0,92,270000.0,90,119500000.0,197200000.0,...,44.0,90,90,95,49,35,85,77,42,19
2,91,94,80.0,95,88.0,91,270000.0,94,45000000.0,83300000.0,...,34.0,90,88,95,29,24,95,86,32,24
3,91,89,86.0,93,94.0,91,270000.0,80,129000000.0,238700000.0,...,37.0,63,93,83,37,35,64,96,32,29
4,91,91,93.0,89,88.0,91,350000.0,91,125500000.0,232200000.0,...,64.0,55,83,82,66,68,63,79,65,53
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19234,47,53,46.0,37,48.0,52,1000.0,46,70000.0,114000.0,...,42.0,48,42,32,42,38,62,55,43,48
19235,47,49,50.0,47,46.0,59,500.0,49,110000.0,193000.0,...,41.0,46,44,33,39,37,47,64,44,47
19236,47,46,45.0,36,49.0,55,500.0,50,100000.0,175000.0,...,41.0,43,39,32,38,38,57,58,44,48
19237,47,48,36.0,47,48.0,60,500.0,48,110000.0,239000.0,...,15.0,40,49,49,14,10,50,72,14,11


In [398]:
row_index = 0
newff_22.loc[row_index, 'overall'] = 92
newff_22.to_csv('updated_dataset.csv', index=False)
newff_22


,overall,movement_reactions,passing,mentality_composure,dribbling,potential,wage_eur,power_shot_power,value_eur,release_clause_eur,...,defending,attacking_heading_accuracy,mentality_penalties,attacking_finishing,mentality_interceptions,defending_marking_awareness,power_jumping,movement_agility,defending_standing_tackle,defending_sliding_tackle
0,92,94,91.0,96,95.0,93,320000.0,86,78000000.0,144300000.0,...,34.0,70,75,95,40,20,68,91,35,24
1,92,93,79.0,88,86.0,92,270000.0,90,119500000.0,197200000.0,...,44.0,90,90,95,49,35,85,77,42,19
2,91,94,80.0,95,88.0,91,270000.0,94,45000000.0,83300000.0,...,34.0,90,88,95,29,24,95,86,32,24
3,91,89,86.0,93,94.0,91,270000.0,80,129000000.0,238700000.0,...,37.0,63,93,83,37,35,64,96,32,29
4,91,91,93.0,89,88.0,91,350000.0,91,125500000.0,232200000.0,...,64.0,55,83,82,66,68,63,79,65,53
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19234,47,53,46.0,37,48.0,52,1000.0,46,70000.0,114000.0,...,42.0,48,42,32,42,38,62,55,43,48
19235,47,49,50.0,47,46.0,59,500.0,49,110000.0,193000.0,...,41.0,46,44,33,39,37,47,64,44,47
19236,47,46,45.0,36,49.0,55,500.0,50,100000.0,175000.0,...,41.0,43,39,32,38,38,57,58,44,48
19237,47,48,36.0,47,48.0,60,500.0,48,110000.0,239000.0,...,15.0,40,49,49,14,10,50,72,14,11


In [399]:
data_22=newff_22.copy()
data_22

,overall,movement_reactions,passing,mentality_composure,dribbling,potential,wage_eur,power_shot_power,value_eur,release_clause_eur,...,defending,attacking_heading_accuracy,mentality_penalties,attacking_finishing,mentality_interceptions,defending_marking_awareness,power_jumping,movement_agility,defending_standing_tackle,defending_sliding_tackle
0,92,94,91.0,96,95.0,93,320000.0,86,78000000.0,144300000.0,...,34.0,70,75,95,40,20,68,91,35,24
1,92,93,79.0,88,86.0,92,270000.0,90,119500000.0,197200000.0,...,44.0,90,90,95,49,35,85,77,42,19
2,91,94,80.0,95,88.0,91,270000.0,94,45000000.0,83300000.0,...,34.0,90,88,95,29,24,95,86,32,24
3,91,89,86.0,93,94.0,91,270000.0,80,129000000.0,238700000.0,...,37.0,63,93,83,37,35,64,96,32,29
4,91,91,93.0,89,88.0,91,350000.0,91,125500000.0,232200000.0,...,64.0,55,83,82,66,68,63,79,65,53
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19234,47,53,46.0,37,48.0,52,1000.0,46,70000.0,114000.0,...,42.0,48,42,32,42,38,62,55,43,48
19235,47,49,50.0,47,46.0,59,500.0,49,110000.0,193000.0,...,41.0,46,44,33,39,37,47,64,44,47
19236,47,46,45.0,36,49.0,55,500.0,50,100000.0,175000.0,...,41.0,43,39,32,38,38,57,58,44,48
19237,47,48,36.0,47,48.0,60,500.0,48,110000.0,239000.0,...,15.0,40,49,49,14,10,50,72,14,11


In [400]:
y=newff_22['overall']
X=newff_22[['overall','potential']]

In [401]:
Xtrain,Xtest,Ytrain,Ytest=train_test_split(X,y,test_size=0.2,random_state=42,stratify=y)

In [402]:
from sklearn.ensemble import RandomForestRegressor
rf_22 = RandomForestRegressor()
rf_22.fit(Xtrain, Ytrain)

RandomForestRegressor()

In [403]:
y_22=rf_22.predict(Xtest)
y_22

array([62., 62., 65., ..., 63., 60., 64.])

In [404]:
mean_squared_error(y_22,Ytest)

1.4033264033261404e-06

In [405]:
dt_22 = DecisionTreeRegressor(random_state=42)
dt_22.fit(Xtrain, Ytrain)

DecisionTreeRegressor(random_state=42)

In [406]:
y_22 = dt_22.predict(Xtest)
y_22

array([62., 62., 65., ..., 63., 60., 64.])

In [407]:
mean_squared_error(y_22,Ytest)

0.0

In [410]:
#finetune decisiontree for fifa22
param_distributions = {
    'max_depth': np.arange(2, 10, step=1),
    'min_samples_split': np.arange(2, 100, step=10),
    'max_features': np.arange(1, 10, step=1)
}
n_iter = 100
random_search_22 = RandomizedSearchCV(
    estimator=DecisionTreeRegressor(),
    param_distributions=param_distributions,
    n_iter=n_iter,
    cv=5
)
random_search_22.fit(Xtrain, Ytrain)


RandomizedSearchCV(cv=5, estimator=DecisionTreeRegressor(), n_iter=100,
                   param_distributions={'max_depth': array([2, 3, 4, 5, 6, 7, 8, 9]),
                                        'max_features': array([1, 2, 3, 4, 5, 6, 7, 8, 9]),
                                        'min_samples_split': array([ 2, 12, 22, 32, 42, 52, 62, 72, 82, 92])})

In [413]:
best_params = random_search_22.best_params_
best22_dt = DecisionTreeRegressor(**best_params)
best22_dt.fit(Xtrain, Ytrain)

DecisionTreeRegressor(max_depth=9, max_features=2)

In [415]:
y_pred = best_dt.predict(Xtest)
mean_squared_error(Ytest, y_pred)

0.0

saving model as pickle file

In [423]:
import pickle
import joblib

model_saved='/content/drive/MyDrive/Colab Notebooks/best22_dt.pk1'
with open(model_saved,'wb')as file:
  pickle.dump(best22_dt,file)

In [424]:
joblib.dump(scaler,'/content/drive/MyDrive/Colab Notebooks/best22_dt.pk1')

NameError: ignored